In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [31]:
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
#         print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except requests.exceptions.ConnectionError: # HTTPError
            re_Dict_gaode["re_location_gaode"] = "请求出错"         
        else:
            if (temp['count'] == "1"):
                if('location' in temp['geocodes'][0]):
                    re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                else:
                    pass
                if ('formatted_address' in temp['geocodes'][0]):
                    re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                else:
                    pass
                if ('province' in temp['geocodes'][0]):
                    re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                else:
                    pass
                if ('country' in temp['geocodes'][0]):
                    re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                else:
                    pass
                if ('city' in temp['geocodes'][0]):
                    re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                else:
                    pass
                if ('district' in temp['geocodes'][0]):
                    re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                else:
                    pass
                if ('level' in temp['geocodes'][0]):
                    re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                else:
                    pass
                if ('township' in temp['geocodes'][0]):
                    re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                else:
                    pass
                if ('adcode' in temp['geocodes'][0]):
                    re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                else:
                    pass
                if ('street' in temp['geocodes'][0]):
                    re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                else:
                    pass
                if ('number' in temp['geocodes'][0]):
                    re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
                else:
                    pass
            elif (temp['count'] == "0"):
                pass
            elif (temp['status'] == "0"):
                pass
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
def parse_gaode(df,address):
    # 解析高德
    list_gaode = get_receiver_lnglat_gaode(address)
    # df化(高德)
    df_gaode = pd.DataFrame(list_gaode)
    data_gaode = pd.concat([df,df_gaode],axis=1)
    print("data_gaode:" + str(data_gaode.shape))
    return data_gaode

In [5]:
# 解析百度地图API网址信息
def get_latlng_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [6]:
# 获取百度地图API地址信息
def get_receiver_lnglat_baidu(address):
    start_time = time.time()
    re_List_baidu = []
    for b in address:
#         print(b)
        re_Dict_baidu = {}
        try:
            temp = get_latlng_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                re_Dict_baidu["re_Lng_baidu"] = temp['result']['location']['lng']
                re_Dict_baidu["re_Lat_baidu"] = temp['result']['location']['lat']
                re_Dict_baidu["re_Precise_baidu"] = temp['result']['precise']
                re_Dict_baidu["re_Confidence_baidu"] = temp['result']['confidence']
                re_Dict_baidu["re_Comprehension_baidu"] = temp['result']['comprehension']
                re_Dict_baidu["re_Level_baidu"] = temp['result']['level']
            else:
                pass
        re_List_baidu.append(re_Dict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：", end_time - start_time)
    return re_List_baidu

In [7]:
def parse_baidu(df,address):
    # 解析百度
    list_baidu = get_receiver_lnglat_baidu(address)
    # df化(百度)
    df_baidu = pd.DataFrame(list_baidu)
    data_baidu = pd.concat([df,df_baidu],axis=1)
    print("data_baidu:" + str(data_baidu.shape))
    return data_baidu

In [8]:
# ak码
# ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_gaode = "1c2682317206ccaa18ec2487bb45d880"
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [9]:
# 导入数据
date = "1902"
fileName = "京东线下(承运商).xlsx"
pinyin = "JDXX"
mainpath = "D:\\work\\basic\\carriers\\"
inputpath = mainpath + date + "\\"
data = pd.read_excel(inputpath + fileName)

In [10]:
# 导出数据路径
outpath_gaode = mainpath + date + "\\output\\gaode\\"
name_ANSI_gaode = date + "Info_lnglat_" + pinyin + "_gaode.xlsx"
name_UTF8_gaode = date + "Info_lnglat_" + pinyin + "_gaode.csv"

outpath_baidu = mainpath + date + "\\output\\baidu\\"
name_ANSI_baidu = date + "Info_lnglat_" + pinyin + "_baidu.xlsx"
name_UTF8_baidu = date + "Info_lnglat_" + pinyin + "_baidu.csv"


In [11]:
data["所在省"].value_counts()

广东     42045
山东     28201
河南     20616
江苏     14692
河北     13475
湖北     12805
北京     10939
湖南     10174
浙江      9755
四川      9663
辽宁      9344
上海      8384
陕西      7868
黑龙江     6934
广西      6105
山西      5956
安徽      5600
云南      5112
吉林      4932
内蒙古     4886
重庆      4018
江西      3996
福建      3579
贵州      3004
甘肃      2695
天津      2562
宁夏       577
海南       530
青海       439
西藏         2
新疆         1
Name: 所在省, dtype: int64

### 选取有效字段

In [12]:
df = data[["所在省","所在市","收货地址","订单编号","承运商","所在县"]]

In [13]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

d:\python\py35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
df1 = df[df["所在省"].isin(["广东"])].reset_index(drop=True)
df2 = df[df["所在省"].isin(["山东"])].reset_index(drop=True)
df3 = df[df["所在省"].isin(["河南"])].reset_index(drop=True)

df4 = df[df["所在省"].isin(["江苏"])].reset_index(drop=True)
df5 = df[df["所在省"].isin(["河北"])].reset_index(drop=True)
df6 = df[df["所在省"].isin(["湖北"])].reset_index(drop=True)
df7 = df[df["所在省"].isin(["北京"])].reset_index(drop=True)
df8 = df[df["所在省"].isin(["湖南"])].reset_index(drop=True)

df9 = df[df["所在省"].isin(["浙江","上海"])].reset_index(drop=True)
df10 = df[df["所在省"].isin(["四川","云南","广西"])].reset_index(drop=True)
df11 = df[df["所在省"].isin(["辽宁","黑龙江","安徽"])].reset_index(drop=True)
df12 = df[df["所在省"].isin(["陕西","山西"])].reset_index(drop=True)

df13 = df[df["所在省"].isin(["吉林","内蒙古","重庆","江西","福建","贵州","甘肃","天津","宁夏","海南","青海","西藏","新疆"])].reset_index(drop=True)

### 文件分组并检查

In [15]:
print("data:" + str(data.shape))
print("df:" + str(df.shape))

data:(258889, 49)
df:(258889, 7)


In [16]:
data1_gaode = parse_gaode(df1,df1["address_all"])

高德总用时： 2610.4536368846893
data_gaode:(42045, 18)


In [17]:
data1_gaode



,所在省,所在市,收货地址,订单编号,承运商,所在县,address_all,re_adcode_gaode,re_city_gaode,re_country_gaode,re_district_gaode,re_formatted_address_gaode,re_level_gaode,re_location_gaode,re_number_gaode,re_province_gaode,re_street_gaode,re_township_gaode
0,广东,深圳市,中山园路建工村保障性住房12栋1105,600101844000008319022502045182,京东线下,南山区,广东省深圳市中山园路建工村保障性住房12栋1105,440305,深圳市,中国,南山区,广东省深圳市南山区建工村保障性住房|12栋,门牌号,"113.927380,22.564155",[],广东省,[],[]
1,广东,潮州市,高堂镇邦湖村,690000000001008319021822065408,京东线下,饶平县,广东省潮州市高堂镇邦湖村,445122,潮州市,中国,饶平县,广东省潮州市饶平县高堂镇,乡镇,"116.940537,23.706640",[],广东省,[],[]
2,广东,韶关市,翁源中学生活区,690000000001008319021822065328,京东线下,翁源县,广东省韶关市翁源中学生活区,440229,韶关市,中国,翁源县,广东省韶关市翁源县翁源中学生活区,兴趣点,"114.127888,24.350587",[],广东省,[],[]
3,广东,广州市,南沙·富力唐宁a10栋302,690000000001008319021822065394,京东线下,南沙区,广东省广州市南沙·富力唐宁a10栋302,440115,广州市,中国,南沙区,广东省广州市南沙区富力唐宁|A|10,门牌号,"113.589013,22.763295",[],广东省,[],[]
4,广东,茂名市,广东茂名市高州市大井镇大井中学,690000000001008319021822065365,京东线下,高州市,广东省茂名市广东茂名市高州市大井镇大井中学,440981,茂名市,中国,高州市,广东省茂名市高州市高州市大井中学,兴趣点,"110.879595,22.095615",[],广东省,[],[]
5,广东,广州市,广东广州市天河区兴盛路6号门岗,690000000001008319021822065341,京东线下,天河区,广东省广州市广东广州市天河区兴盛路6号门岗,440106,广州市,中国,天河区,广东省广州市天河区兴盛路,道路,"113.331512,23.120383",[],广东省,[],[]
6,广东,广州市,广东广州市增城市荔城街开园路9号,690000000001008319021822065316,京东线下,增城市,广东省广州市广东广州市增城市荔城街开园路9号,440118,广州市,中国,增城区,广东省广州市增城区开园路|9,门牌号,"113.827860,23.293701",9,广东省,开园路,[]
7,广东,肇庆市,凯悦华庭金龙湾二楼前台,690000000001008319021822065314,京东线下,端州区,广东省肇庆市凯悦华庭金龙湾二楼前台,441202,肇庆市,中国,端州区,广东省肇庆市端州区金龙湾,兴趣点,"112.450437,23.052380",[],广东省,[],[]
8,广东,肇庆市,凯悦华庭金龙湾二楼前台,690000000001008319021822065309,京东线下,端州区,广东省肇庆市凯悦华庭金龙湾二楼前台,441202,肇庆市,中国,端州区,广东省肇庆市端州区金龙湾,兴趣点,"112.450437,23.052380",[],广东省,[],[]
9,广东,广州市,广东广州市白云区广州市白云区白云山南门,690000000001008319021822065292,京东线下,白云区,广东省广州市广东广州市白云区广州市白云区白云山南门,440111,广州市,中国,白云区,广东省广州市白云区白云山,道路,"113.304462,23.163476",[],广东省,[],[]


In [18]:
data2_gaode = parse_gaode(df2,df2["address_all"])

高德总用时： 3320.1223702430725
data_gaode:(28201, 18)


In [19]:
data3_gaode = parse_gaode(df3,df3["address_all"])

高德总用时： 1151.599315404892
data_gaode:(20616, 18)


In [20]:
data4_gaode = parse_gaode(df4,df4["address_all"])

高德总用时： 850.4858589172363
data_gaode:(14692, 18)


In [21]:
data5_gaode = parse_gaode(df5,df5["address_all"])

高德总用时： 841.8395233154297
data_gaode:(13475, 18)


In [22]:
data6_gaode = parse_gaode(df6,df6["address_all"])

高德总用时： 778.1571390628815
data_gaode:(12805, 18)


In [23]:
data7_gaode = parse_gaode(df7,df7["address_all"])

高德总用时： 700.0656762123108
data_gaode:(10939, 18)


In [24]:
data8_gaode = parse_gaode(df8,df8["address_all"])

高德总用时： 663.4431774616241
data_gaode:(10174, 18)


In [25]:
data9_gaode = parse_gaode(df9,df9["address_all"])

高德总用时： 1028.0044448375702
data_gaode:(18139, 18)


In [30]:
data10_gaode = parse_gaode(df10,df10["address_all"])

广西省南宁市高新区总部路1号东盟企业总部基地1期A1栋2楼
四川省成都市崇州市街子镇会元村十四组
四川省南充市四川南充市南部县黄家坝幸福二巷67号韵达快递
四川省广安市邻水县柑子镇
云南省曲靖市太和街322号(文庙牌坊旁)
四川省南充市红岩子大道552号
云南省玉溪市云南玉溪市江川县路居镇上坝梅竹村
四川省南充市南充市顺庆区金台镇
四川省成都市东郊记忆32号楼火车头广场3楼今日头条空间
四川省成都市郫都区高新青年公寓六号苑
云南省昆明市雨花街道云南省医药有限公司物流中心李庚运收
广西省柳州市航生路东苑小区5区63号
四川省绵阳市科学城三区六号楼三单元101
广西省柳州市宏桂香兰花园2栋一单元10-4
广西省南宁市秀灵路西二里秀厢村八组聚秀园小区C3-14
四川省成都市南街200号中国银行门卫处
云南省昆明市万宏路与联盟路交叉口万宏国际一期2栋701
四川省南充市吊索湾村四组43号
云南省普洱市宁洱大道245号
云南省昆明市云南昆明市官渡区关上街道和甸营村
四川省南充市黑水塘幸福小区3栋2单元2楼4号
广西省北海市独树根东一巷二号
四川省南充市民乐街7号
四川省德阳市建大家园(云峰山路)
广西省贵港市广西贵港市湛江镇
广西省南宁市佛子岭路18号德利菜鸟驿站
云南省大理白族自治州云南大理白族自治州弥渡县新街镇陶家营
广西省崇左市城北路万达停车场内
云南省昆明市云南昆明市盘龙区玉器城翡翠花园1组团6栋3单元502
四川省成都市四川成都市郫县红光镇学府南路8号天邑宏御花园5栋1单元1-10-2室
广西省玉林市大坡外新街
四川省眉山市瑞能国际·光伏科技园2栋1单元
云南省红河哈尼族彝族自治州个旧市小平台1拣
广西省梧州市湄江北路52号
广西省桂林市日头火桂林米粉(阳朔店)
四川省绵阳市塔水镇双埝村4队
广西省贺州市广西贺州市富川瑶族自治县城北镇中通快递
云南省红河哈尼族彝族自治州宝华小区一心堂药店
四川省成都市柏杨小区1期
四川省绵阳市人民街46号2栋2单元401号
广西省柳州市乐和大夏34楼3418
四川省乐山市大田街130号13幢3单元3楼1号
四川省成都市四川农业大学
四川省宜宾市莲花镇高店村。
四川省眉山市阳光世纪风景
四川省南充市四川南充市南部县凌云路106号
广西省百色市广西百色市靖西县信发铝厂
云南省昭通市昭阳区爱民路130附3号
云南省西双版纳傣族自治州云南省

广西省河池市广西省河池市宜州区石别镇板围村板围屯116号
四川省遂宁市九莲街道明月路君豪阳光酒店2楼爱尚咖啡
广西省柳州市百饭路46号10栋一单元
四川省资阳市简乐桥村六组
云南省大理白族自治州下庄街顺丰快递收
四川省绵阳市绵阳市游仙区小枧沟镇惠明街红日厨卫
四川省达州市南外镇伴山名居B2幢8号库房
云南省文山壮族苗族自治州云南文山壮族苗族自治州麻栗坡县麻栗坡莱溪简尚
四川省成都市一环路东一段251号
广西省桂林市仕通路69号
四川省巴中市南坝太子路南段
广西省桂林市临桂区世纪东路月亮湾小区1栋3单元401
广西省梧州市新兴一路3一号梧州市红十字会医院急诊科
四川省自贡市四川自贡市荣县梓洞桥村十组
广西省崇左市广西壮族自治区崇左巿江州区石景林街道广西城市职业学院崇左校区
四川省成都市天回镇土门村三组舒鑫旅馆二搂13号
四川省达州市欧家镇高家街道沙河路151号
广西省桂林市广西桂林东安街176
广西省贺州市西约街竹篙巷32号
云南省昆明市云大西路云电科技园云南电力技术有限公司
广西省南宁市亭洪路广西广播电视学校
四川省南充市四川省仪陇县永乐镇仙鹤160号符89号
四川省成都市灌口镇蒲阳路1068号國际名都6幢l单元307
广西省柳州市柳长路一区十栋二单元2-2室
广西省桂林市中仁居民区B4栋16号
广西省南宁市广西南宁市江南区淡村三里13栋楼（雅堂小超）
四川省内江市严陵镇广场巷117号
广西省贵港市广西壮族自治区贵巷市县官成镇朝新村朝垌屯一40号
四川省泸州市沙坎上房村十三社
广西省桂林市临桂区金山路132号
广西省桂林市临桂区′路口村54号韵达炔递
广西省桂林市临桂区金水路东坡园15栋18号
广西省柳州市红卫路29号车龙汽车装饰
四川省巴中市南江县红塔一社安置房
广西省桂林市广西省桂林市临桂区仕通路69号
云南省昆明市060乡道山脚村云南健阳农机有限公司
云南省昆明市云南省昆明市西山区海口镇蒋凹老村
云南省昆明市海源社区秀才塘新村67号(鑫都宾馆)
广西省玉林市白石桥路259号
四川省乐山市凤凰路中段205号
广西省南宁市广西南宁市兴宁区畅春湖山庄A区19号
云南省文山壮族苗族自治州文山市薄竹镇
广西省贺州市钟山镇塘桥村003号
四川省南充市四川南充市西充县多扶镇丽景国际
广西省桂林市广西桂林市雁山区广西桂林市雁山区柘木镇李家村
广西省崇左市龙门乡龙内屯5号
广西

广西省南宁市广西南宁市麻村二街111号
四川省成都市三里花城项目部
四川省雅安市汉源二中
四川省绵阳市塘汛吉祥宏达超市
云南省昆明市昆明市双桥村711号
广西省玉林市田家炳中学对面丰润超市
四川省成都市来福士广场负二楼
广西省贵港市大岭乡30队
四川省南充市四川省南充市高坪区东岸十九座
四川省成都市四川成都市青羊区战旗东路9号
云南省昆明市马金铺镇生物谷街511号明汇光学
四川省成都市应龙路1199号
四川省乐山市四川乐山市市中区肖坝路肖坝街222号成都理工大学工程技术学院
四川省成都市美年广场c座368
广西省柳州市寿星中路11号融水中学
云南省玉溪市华宁县盘溪镇华宁二中
云南省昆明市中国石油月马路加油站
云南省曲靖市福佑药店(光华街)
广西省桂林市翠竹路南巷12号万正西区国际3一2一2一1
广西省南宁市琅东一组11栋6号
广西省南宁市广西南宁市江南区明阳工业园区广西演艺学院
云南省昆明市富民工业园区哨箐机械加工园富民文楚农机植保厂
广西省南宁市广西南宁市兴宁区三塘镇九曲湾大门
云南省昆明市吴井街道拓东大成二期6栋
四川省广安市崇文大道山水丽都2栋4单元5-1
四川省成都市大面青台绿阁4栋二单元2605
广西省来宾市广西壮族自治区柳州市柳州市柳江区柳东路121号柳州商贸技工学校
四川省宜宾市四川宜宾市翠屏区上江北古塔路阳光苑
广西省桂林市桂林市公安局出入境管理处
云南省临沧市中医院住院部3楼
云南省曲靖市云南能源职业技术学院
四川省宜宾市南岸鑫杰座放物管处
广西省柳州市三江县古宜雅谷路安昌商行
云南省大理白族自治州鹤庆县松桂镇
广西省南宁市广西南宁市青秀区桂春路2号中国移动南宁分公司2号楼4F
云南省大理白族自治州大理万福汽车销售服务有限公司
广西省贵港市平山镇平容街102号(恒大驾校隔离)
四川省泸州市金山路美领居一期
云南省临沧市新兴街云雪加油站背后兴兴家居
四川省成都市古城镇
广西省贵港市广西桂平市西山二中门口
云南省文山壮族苗族自治州学府路文山学院
云南省临沧市晓街乡晓街供电所
广西省桂林市人民路鑫源广场一栋5一4号
广西省南宁市广西南宁市西乡塘区大岭村爱心幼儿园附近
广西省南宁市长堽路二里7号南宁中南理工职业技术学校
四川省广元市九龙镇邮政局
云南省昆明市浪泥湾(新村)
广西省贵港市广西贵港市港北区大将国际学校1704班
广西省玉林市广西博白县宝中宝超市

广西省贺州市梅花福堂村
云南省西双版纳傣族自治州嘎栋工业园区南溪大道6号郑州日产乐景店
广西省桂林市临政路天鹅堡小区六栋17-1
云南省曲靖市云南曲靖市马龙县旧县镇
广西省桂林市灌阳镇解放西路29号
云南省曲靖市廖郭北路河北商业街学府公寓旁229号
四川省成都市天府街道天府智谷E栋一单元
广西省桂林市临政路天鹅堡小区六栋17-1
四川省成都市双建路387号师徒情深人民食堂
四川省绵阳市四川绵阳市涪城区九洲大道88号中华坊
广西省南宁市衡阳街道友爱路西三巷（小摇蓝幼儿园）
四川省成都市北泉路与玉龙街交叉口西南100米龙泉驿都城2期
四川省泸州市四川省泸州市龙马潭区安宁镇安居街52号
广西省玉林市广西壮族自治区博白县实验中学
云南省曲靖市云南省曲靖市麒麟区紫云路东星小区旁化工基地
广西省玉林市广西玉林市陆川县乌石镇平山白土二队46号
云南省普洱市龙旺井12号
广西省桂林市桂林市象山区交通路15号
四川省绵阳市绵阳市涪城区花园南街50鼎鑫大厦13-2
云南省昆明市气象路32号4幢3单元303
广西省河池市城东站钻石豪庭门卫处
四川省成都市四川成都市成华区热电厂生活区31栋5单元2楼
四川省内江市东兴区。晨望路。中和新都汇保安室
云南省昆明市滇池度假区滇池路1045号尚欧国际家居
云南省昆明市气象路32号
云南省西双版纳傣族自治州云南省景洪市东风农场一分场
广西省贵港市桂平市木根镇宁垌村
广西省崇左市广西崇左市大新县桃城镇上贯屯
云南省楚雄彝族自治州团结路金山花园B栋309室
四川省乐山市杨柳镇杨柳村中铁上海局成贵铁路项目部
云南省普洱市土桥心联兴超市对面卖部
广西省桂林市桂林市临桂县临桂区虎山路金狮港猫儿山58号
四川省成都市芳草东街一号院
云南省红河哈尼族彝族自治州五一路五一东村3幢403号
广西省桂林市临桂镇环湖路口
云南省昭通市白鹤滩镇花桥社区过境路86号
四川省宜宾市四川宜宾市翠屏区南岸西区宜宾市职业技术学校（大湾校区）
云南省昆明市云南昆明市盘龙区蓝龙潭四水厂后迪固石英石厂
四川省宜宾市四川宜宾市翠屏区白沙湾街道地中海蓝湾蓝湾佳美酒店
四川省成都市公平街时代西锦3号门
四川省成都市大弯，黄金路201号，安捷汽修
四川省成都市量力商务大厦1204号
广西省南宁市武鸣区国土局
云南省昆明市云南省昆明市呈贡区惠兰园小区
广西省梧州市金鸡镇金鸡水利枢纽
四川省广元市四川

广西省贺州市鞍山路西路83号4-2单元503
广西省钦州市钦州钦南区北部湾职业技术学校
四川省成都市天回镇土门新村4组
广西省河池市教育路能源办对面飞环摩配
四川省达州市万源市职业高级中学（万兴路西）秀秀文具店
四川省广安市广安市广安区城南上甲山水快递箱
四川省成都市四川成都市锦江区世纪朝阳5-1-2201
云南省西双版纳傣族自治州云南省西双版纳傣族自治州景洪市大猛龙镇东风农场东风花园小区
四川省广安市广安职业技术学院
广西省玉林市广西玉林市兴业县塔竹平村
四川省泸州市四川省叙永一中
广西省玉林市广西玉林市兴业县塔竹平村
四川省成都市四川省成都市新都区电子路99号
四川省成都市东山国际C区
广西省崇左市柳桥镇渠齐村岜亿屯48号
云南省大理白族自治州恒通鞋业
四川省成都市圃园南二路10号凌远花园
四川省广安市东城首座
四川省攀枝花市西区建校
四川省绵阳市四川绵阳市涪城区跃进路五号跃进路幼儿园
广西省百色市广西工程职业学院（平果校区）
广西省河池市城关镇中平村上村屯02号
广西省桂林市栗木镇大合渡船头村
四川省成都市和心路499号
云南省大理白族自治州下庄镇联民副食
广西省柳州市广西柳州市柳南区南站街道柳工颐华城1栋1-3号YY文具精品店
云南省红河哈尼族彝族自治州凤麟街54号
广西省北海市广西北海市海城区东街街道宝宜世界负一楼
四川省成都市和心路499号
云南省楚雄彝族自治州西苑路127号大姚第一中学
云南省昆明市云南省昆明市盘龙区茨坝街道永都酒店
四川省宜宾市南岸江源半岛九栋二十四楼五号
四川省成都市四川成都市青羊区顺城大街27号嘉好大厦1417
云南省楚雄彝族自治州共和镇万寿路化湖人家正大门值班室
四川省成都市四川成都市金堂县淮口工业园华美保温材料有限公司
四川省成都市四川省成都崇州市机电技师学院
四川省成都市新盛路469号新和领航2栋920A
云南省曲靖市云南省曲靖市麒麟区珠江源广场休闲步行街36号菩提树下
云南省红河哈尼族彝族自治州个旧市乍甸
四川省南充市南部中学东区
云南省昆明市万兴印象小区2栋2单元701室
四川省成都市四川省成都崇州市机电技师学院
四川省凉山彝族自治州交警队旁财政局家宿院
云南省楚雄彝族自治州曙光路彩虹街111号清真牛菜馆
云南省昭通市北新家园公租房
云南省大理白族自治州中红海峡广场4楼高品会牛排自助餐厅
广西省南宁市广西南宁市江南区横岭

...

广西省桂林市八里街金家村65号
四川省内江市文兴街110号
云南省昆明市泛亚城邦二栋
四川省自贡市汇东新区汇西路白果小区10栋2单元713
四川省乐山市四川省乐山市延风中学
四川省成都市新南门地铁站口
广西省河池市解困小区
四川省自贡市四川自贡市自流井区汇东新区汇西路白果小区10栋2单元713
广西省贵港市贵港卫生学校新校区
云南省大理白族自治州祥云县云南驿镇前所
云南省文山壮族苗族自治州树皮乡树皮街上
云南省文山壮族苗族自治州文山州马关县马白镇骏城路华联生态一期木兰苑A幢11号贤嘉丽面包店
四川省巴中市高明职业中学
广西省柳州市广西柳州市柳北区跃进路东二巷79号
广西省桂林市桂林市、七星区、自由路、郑远元专业修脚房
四川省乐山市绥山镇清音路72号
云南省西双版纳傣族自治州云南省西双版纳景洪市白象花园一区二单元201
云南省昭通市云南省昭通市威信县扎西镇长地办事处沙子坝社村民小组
四川省成都市晋吉南路22号21栋3单元1楼1号
四川省绵阳市中坝镇会昌巷1号1幢3单204
云南省昆明市金马寺水木山居3栋2单元301号
四川省巴中市西城街道印合路林海居三期
四川省成都市西航港通关路122号四川中移通信技术工程有限公司新地库区(西南门)
四川省广元市四川省广元市利州区利州西路二段179号
广西省南宁市广西南宁市邕宁区四十二中
四川省成都市川庆钻探工程公司井下作业公司(华盛路)
四川省宜宾市特巡警大队
四川省眉山市文林镇安置二街天义洗车场对面
广西省河池市广东省广州市从化区良口镇从都国际庄园（后勤部）
云南省曲靖市天生坝十二村72号
四川省乐山市犍为县玉津镇圣泉路385号
四川省内江市环城路和平街361号
云南省西双版纳傣族自治州大猛龙填东风农场东风花园小区
云南省曲靖市西宁路152号附6号（顺馨商店旁）
四川省自贡市罗家湾六栋
广西省来宾市城关镇忻城县中学（县中）
四川省乐山市帮泰国际社区7栋17楼4号
云南省昆明市云南省昆明市嵩明县杨林镇老余屯村5号
云南省丽江市民贸小区C懂一单元
四川省成都市黄忠路惠民苑14栋1单元
云南省楚雄彝族自治州新村镇巨龙村中通快递公司
广西省贵港市麻垌镇荔城街
云南省昆明市匡远街道花园街中通四分部
云南省西双版纳傣族自治州白象巷12号聚仁兴宾馆
广西省百色市万元街绿色动力网吧
云南省文山壮族苗族自治州砚山县布标工业园区
广西省南宁市广西南宁

云南省昭通市集英小镇5栋
四川省成都市华成宾馆对面东方超市
广西省玉林市广西玉林市博白县博白县实验中学
广西省桂林市瓦窑街道平山公交站益丰苑7-2-201
四川省南充市和平东路半环巷
云南省临沧市洗布巷15号
云南省昭通市昭通市昭阳区东煤路骋宇驾校门口
广西省桂林市滨江路香脆鱼馆
四川省雅安市百步街二娃饭店
云南省曲靖市云南省曲靖市麒麟区三岔社区牛街32栋
云南省西双版纳傣族自治州景洪市曼听路29号油乡商务酒店旁大渔塘
云南省昆明市官渡区高原明珠美食街B--3
四川省成都市货运大道玉柴机器689号维修接待大厅
云南省昭通市昭通市昭阳区苏家院乡中通快递
云南省昆明市福海乡街道船房新村和平商店108号
云南省玉溪市云南省玉溪市红塔区下载井2栋9号
云南省昆明市东风东路115好号
四川省凉山彝族自治州盐源中学校
广西省北海市广西北海市银海区侨港镇金海岸大道胜境宾馆
云南省昆明市金瓦路云南第三机床厂宿舍16幢1单元306
广西省梧州市黄村镇卫生院
广西省梧州市新兴二路恒祥花苑一期7幢3单元801房
四川省成都市成都铁路工程学校
四川省达州市红旗路时代天骄华庭A栋7楼1号
广西省桂林市凯风路80号御林弯20栋603室
云南省昆明市云南省昆明市禄劝县屏山镇南街道办事处办事处五星50号
四川省泸州市红星街道美领居3期1栋1310
广西省南宁市横州镇茉莉花大道北一巷9号
广西省北海市四号路诚德镍业有限公司
四川省内江市资中二中教师公寓
云南省昆明市大板桥东强纸业前行200米门口超市
云南省文山壮族苗族自治州云南省文山州广南县家和莲郡十幢一单元102
广西省南宁市黎塘镇永安西一里12号
云南省昆明市嵩明县四营街下马坊村
广西省柳州市雒容镇九龙寨
广西省南宁市广西南宁市兴宁区广西交通职业技术学院
四川省广安市玉兔路上甲山水盛景2单元202
广西省桂林市苏桥镇水荆西路苏桥经济开发区苏桥工业园长海科技桂林疗养院中建八局项目部
广西省桂林市彰泰公园一号1栋
四川省南充市四川省南充市顺庆区西华路二段89号（南充市特殊教育学校）
云南省昆明市云南省电子信息高级技工学校
广西省桂林市金水路香樟林小区
四川省泸州市金科博翠湾总包办公室
广西省百色市华银二期13栋4单元504号
四川省成都市四川省成都市锦江区锦东路253号院一单元302
四川省成都市四川成都市新都区龙桥镇河顺街宏发五金店
四川省泸州市

云南省曲靖市麒麟区曲靖市西城街道西山邮政所
云南省大理白族自治州弥渡汽车客运站
广西省北海市吉林路11号工业园善水湾3-3-303
四川省巴中市通江县诺江中路诺水云天B栋406号
四川省巴中市南坝社区居委会
四川省雅安市友谊路四川省档案学校
四川省成都市四川省成都市双流区蛟龙港新川藏路720成都浩丰销售有限公司
广西省玉林市广西玉林市博白县博白县第三高级中学
云南省曲靖市花柯新村58号
四川省成都市清波路385号“金沙愉景二期”
云南省普洱市茶城大道66号2栋601
四川省自贡市四川省自贡市荣县荣逸宾馆
云南省曲靖市云南曲靖市沾益县西平镇云南工业技师学院
云南省红河哈尼族彝族自治州建水县临安镇711信箱600号
广西省贵港市广西贵港市港北区西江绿城西江园路狮子岭一小区十八幢二号
广西省百色市广西百色市右江区大华路8号
云南省文山壮族苗族自治州文山市普阳东路盘龙河畔小区A17栋B座
广西省百色市百胜坡射巷27号
四川省德阳市向阳镇幸福小区
云南省昆明市雨龙路和甸营村
四川省南充市四川南充市西充县晋城镇建设路老检察院
广西省梧州市藤州镇同乐四街11号
四川省德阳市四川德阳市什邡市什邡中学高中部家属区
广西省柳州市解放街322号
广西省桂林市时代花园27栋
云南省昭通市金融中心座
广西省百色市广西百色市右江区大华路8多
云南省文山壮族苗族自治州文山县德厚镇，菲古小塘子村
四川省泸州市兰田镇立行路机麻茶园
广西省百色市广西百色市右江区大华路8号
四川省成都市四川省成都市武侯区高升桥22号海友酒店
广西省贵港市西山镇长安工业区玮力裕科技园建颖厂
四川省凉山彝族自治州四川省西昌市开元乡4组
广西省来宾市自取
四川省广安市白市镇小学对面阳光家园三楼
四川省成都市郭家桥南街3号2-3-9
云南省曲靖市陆良县罗家小街中通快递代收
云南省昭通市昭阳区凤凰街道元宝山体育运动学校
云南省昆明市蓝光天骄城
云南省昆明市东风东路47号建业商务中心B座10楼口佳家
广西省百色市广西百色市右江区大华路8号
云南省昆明市永定街道办事处永定街款庄小区2幢
云南省昭通市粮食局柒琦大酒店对面艾尚美发沙隆
云南省文山壮族苗族自治州云南省文山州麻栗坡县麻栗镇民族职业高级中学
四川省绵阳市科学城五区世纪华联超市一楼红红家纺
四川省成都市金鹏街175号3栋3单元1号
广西省百色市广西百色市右江区大华路8号
四川省

云南省保山市隆阳区保山技师学院菜鸟驿站
云南省昆明市珥季路后所村
四川省宜宾市四川宜宾市宜宾县柏溪镇裕川翠柏小区一楼门市惠丰电脑斜对面
四川省广元市川北幼儿示范高等专科学校
四川省宜宾市顺江村八组14号
四川省绵阳市虹苑(长虹第一职工宿舍)
云南省玉溪市云南省玉溪市红塔区东风北19号
云南省昆明市乡鸭湖小区观澜苑2幢2单元
云南省大理白族自治州府前街7号
云南省昆明市菜鸟驿站(昆明星汇园D6栋1层64号店)
广西省南宁市大丰镇古洞路47号
四川省成都市东山国际青年广场37栋
云南省昆明市宜良县城内
四川省达州市龙头桥村1组
四川省宜宾市来复812厂江南客运队
云南省昆明市云南昆明市盘龙区龙泉路云南机电职业技术学院菜鸟驿站
广西省玉林市广西玉林市博白县兴隆街邮电局宿舍第一栋501号
四川省巴中市四川巴中市巴州区将军大道，丽景酒店E，秋金
四川省泸州市红锦玻璃集团
广西省梧州市南渡镇桥东街义新五金店
广西省桂林市环城北二路36号冠泰水晶城32栋
广西省柳州市新柳大道99号鹿山学院
四川省广元市四川广元市利州区广元市利州区工农镇汤山女皇温泉酒店
四川省成都市四川成都市金牛区育仁东路二十号福祥家园瑞福苑
云南省昆明市云南昆明市官渡区大板桥泉鑫酒店
云南省曲靖市街兴花园4路，6路旁易点通讯
四川省自贡市富世镇顺城街269号
四川省成都市中大君悦金沙7期7栋
四川省宜宾市景苑小区4幢1703
云南省德宏傣族景颇族自治州泰安路3号
四川省内江市康复中路481号朝荣食府
广西省南宁市药用植物园龙翔荆楚高中
广西省贵港市大有化肚村
云南省昭通市盐井镇坪街84号盐津县第一中学校
广西省梧州市藤州中学
广西省南宁市广西大学西三栋211
四川省成都市北京华联生活超市(空港购物中心店)五楼爱客得
四川省成都市四川成都市双流县天府大道南段2036号
云南省文山壮族苗族自治州云南文山壮族苗族自治州文山县职教园区
云南省昭通市高滩新区国际花园6栋二单元4楼5号
四川省成都市董场镇正雄社区8组
四川省成都市四川成都市彭州市丹景山镇丹寿苑(申通快递代收)
广西省桂林市桂林市，临桂区会元路111号
云南省楚雄彝族自治州雍和居4栋904
云南省昆明市民胜家园西区5幢5单元
云南省西双版纳傣族自治州云南西双版纳傣族自治州勐海县八公里曼短村锦茗茶厂
云南省昆明市云南昆明市官渡区新华丰c区5栋4号
四川省凉山彝

四川省成都市果壳里的城
广西省钦州市灵山县平南镇百世快递
云南省楚雄彝族自治州禄丰县大华街
云南省红河哈尼族彝族自治州蒙自市新安所镇
广西省玉林市东平镇
云南省普洱市杨文景云南省普洱市宁洱县中通快递
广西省南宁市北湖路万秀村三队土地庙申通驿站
云南省保山市沙坝村三组。
四川省甘孜藏族自治州新榆林家中
云南省昆明市西山区昆明工业学校。
云南省怒江傈僳族自治州兰坪县营盘镇
云南省昆明市昆明市禄劝县撒营盘镇
广西省河池市都安瑶中
四川省成都市四川省邛崃市金丰宜居11栋2单元6号
广西省来宾市迎宾广场金塘路26姐妹服装店
广西省贵港市立志中学医院后背7号牌
四川省达州市欧家镇高家街道沙河路12号
广西省钦州市永福西大街96号宁越花园11-2-1801
广西省百色市百都乡果庇村坡松屯
四川省南充市安汉大道29号亿林龙城十栋一单元
广西省河池市南环路81号
云南省楚雄彝族自治州楚雄职教中心
广西省南宁市荔滨大道28号区党校南门保利君悦湾东阳三建项目部
四川省眉山市三苏大道七十二号
云南省保山市兰城路和平小区75号韩妃美容美体养生馆
四川省绵阳市城郊乡金家林金林家园
四川省成都市德源镇菁蓉镇大禹东路175号新城佳苑
四川省成都市四川省成都市锦江区花果新居二期
云南省昆明市洛羊街道石龙路云南现代职业技术学院
广西省桂林市奇峰镇8号40栋
四川省乐山市苏稽计算机学校
四川省绵阳市城郊乡金家林金林家园一单元
云南省楚雄彝族自治州云南省楚雄州武定县插甸镇依纳格村
云南省昆明市经开区国际银座紫曼园小区一栋1102
四川省巴中市下两镇，下两新街176号
四川省成都市驿都西路775号成都市武警支队旁，洪河茶业公司
广西省贵港市广西壮族自治区贵港市港北区贵城街道贵港市白云技术学校
云南省昆明市矣六办事处渔村808
广西省桂林市九屋申通快递
广西省梧州市广西梧州市蒙山县湄江街北路东138号
四川省达州市欧家镇三溪村
四川省宜宾市安埠街道吊黄楼社区2组68号
云南省昆明市晋宁县上蒜鎮河泊村委会
广西省南宁市沙井镇南乡新村三组二栋3号
广西省百色市广西百色市西园路林业局宿舍
云南省昆明市逸康路花都花园B区4栋3单元1202室
云南省昆明市云南省昆明市宜良县清远街双桥路十号
广西省玉林市众宇新城18栋1单元602室
四川省广安市街子镇安置房一期B幢2单元602
云南省昆明市云南省昆明市宜良县清远街双桥路

In [32]:
data11_gaode = parse_gaode(df11,df11["address_all"])

高德总用时： 1216.7100462913513
data_gaode:(21878, 18)


In [33]:
data12_gaode = parse_gaode(df12,df12["address_all"])

高德总用时： 798.1982612609863
data_gaode:(13824, 18)


In [34]:
data13_gaode = parse_gaode(df13,df13["address_all"])

高德总用时： 1888.3507454395294
data_gaode:(31221, 18)


In [50]:
data_gaode = pd.concat([data1_gaode,data2_gaode,data3_gaode,data4_gaode,data5_gaode,data6_gaode,data7_gaode,
                        data8_gaode,data9_gaode,data10_gaode,data11_gaode,data12_gaode,data13_gaode])

In [40]:
data_gaode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258889 entries, 0 to 31220
Data columns (total 18 columns):
所在省                           258889 non-null object
所在市                           258889 non-null object
收货地址                          258889 non-null object
订单编号                          258889 non-null object
承运商                           258889 non-null object
所在县                           258889 non-null object
address_all                   258889 non-null object
re_adcode_gaode               258611 non-null object
re_city_gaode                 258611 non-null object
re_country_gaode              258611 non-null object
re_district_gaode             258611 non-null object
re_formatted_address_gaode    258611 non-null object
re_level_gaode                258611 non-null object
re_location_gaode             258880 non-null object
re_number_gaode               258611 non-null object
re_province_gaode             258611 non-null object
re_street_gaode               258611 non-n

In [51]:
data_gaode['re_location_gaode'] = data_gaode['re_location_gaode'].map(lambda x:str(x))

In [60]:
# 合表
split_gaode = pd.DataFrame((x.split(',') for x in data_gaode['re_location_gaode']), index=data_gaode.index, columns=['re_siteLng_gaode','re_siteLat_gaode'])



In [68]:
# data_gaode1 = pd.merge(data_gaode,split_gaode,left_index=True,right_index=True)
data_gaode2 = pd.concat([data_gaode,split_gaode],axis=1)

In [53]:
# 检查合并后结构
print("data:" + str(data.shape))
print("df:" + str(df.shape))
print("data_gaode1:" + str(data_gaode1.shape))
# print("data_baidu:" + str(data_baidu.shape))

data:(258889, 49)
df:(258889, 7)
data_gaode1:(2566627, 20)


In [71]:
data_gaode2.to_csv(outpath_gaode + name_UTF8_gaode,index=False)
# data_baidu.to_csv(outpath_baidu + name_UTF8_baidu,index=False)

In [66]:
data_gaode1.index

Int64Index([    0,     0,     0,     0,     0,     0,     0,     0,     0,
                0,
            ...
            42035, 42036, 42037, 42038, 42039, 42040, 42041, 42042, 42043,
            42044],
           dtype='int64', length=2566627)

In [65]:
split_gaode.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            31211, 31212, 31213, 31214, 31215, 31216, 31217, 31218, 31219,
            31220],
           dtype='int64', length=258889)

In [67]:
data_gaode.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            31211, 31212, 31213, 31214, 31215, 31216, 31217, 31218, 31219,
            31220],
           dtype='int64', length=258889)

In [62]:
data_gaode.re_location_gaode.isnull().value_counts()

False    258889
Name: re_location_gaode, dtype: int64

# 

In [70]:
data_gaode2.shape

(258889, 20)